In [22]:
import tensorflow as tf
import numpy as np

## Construction Phase

In [23]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 =100
n_outputs = 10 

In [24]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")

 - Creating subgraph for one neuron layer 

In [25]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X,W) + b 
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

#### Breakdown of neuron_layer( ) code

1. Creating a name scope using the name of the given layer 
2. Getting the number of inputs
3. Initializing wieghts by a Gaussian Distribution
4. Initializing biases with zero
5. Creating a subgraph to compute $z = X.W +b$
6. Returning output based on activation function


- Creating graph for a deep neural network with 2 hidden layer

In [26]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

- Subgraph for computing loss

In [27]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                             logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

- Subgraph for optimization using gradient descent 

In [28]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

-  Subgraph for evaluation metrics

In [29]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Now let's initialize

In [30]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [31]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

## Execution Phase

In [32]:
# loading the dataset

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [33]:
n_epochs = 40
batch_size = 50 

In [34]:
# Training the model 

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                           y: mnist.test.labels})
        print(epoch, 
              "Train Accuracy:", acc_train, 
              "Test Accuracy:",acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train Accuracy: 0.88 Test Accuracy: 0.9146
1 Train Accuracy: 0.98 Test Accuracy: 0.9286
2 Train Accuracy: 0.96 Test Accuracy: 0.9383
3 Train Accuracy: 0.94 Test Accuracy: 0.9435
4 Train Accuracy: 0.92 Test Accuracy: 0.9486
5 Train Accuracy: 0.98 Test Accuracy: 0.9523
6 Train Accuracy: 0.98 Test Accuracy: 0.9529
7 Train Accuracy: 0.96 Test Accuracy: 0.9574
8 Train Accuracy: 1.0 Test Accuracy: 0.9594
9 Train Accuracy: 1.0 Test Accuracy: 0.9623
10 Train Accuracy: 0.98 Test Accuracy: 0.964
11 Train Accuracy: 1.0 Test Accuracy: 0.9649
12 Train Accuracy: 0.98 Test Accuracy: 0.9659
13 Train Accuracy: 0.94 Test Accuracy: 0.9656
14 Train Accuracy: 0.96 Test Accuracy: 0.9677
15 Train Accuracy: 0.98 Test Accuracy: 0.9679
16 Train Accuracy: 1.0 Test Accuracy: 0.9688
17 Train Accuracy: 0.98 Test Accuracy: 0.969
18 Train Accuracy: 0.98 Test Accuracy: 0.9713
19 Train Accuracy: 1.0 Test Accuracy: 0.9708
20 Train Accuracy: 0.98 Test Accuracy: 0.971
21 Train Accuracy: 1.0 Test Accuracy: 0.972
22 Train